# Window functions

- 앞서 배웠던 pyspark.functions는 substr()과 같이 문자열을 잘라내고, 그 결과를 생성하여 행으로 적용한다.

- 반면에 윈도우 함수는 그룹으로 구분하고 그 범위 내에서 계산을 할 때 사용한다.

- 이 때 over() 함수로 적용되는 그룹의 윈도우를 구분하게 된다. 

#### 순위함수
- ranking functions : rank, dense_rank, percent_rank, ntile, row_number

#### 분석 함수
- analytic functions : cume_dist, first_value, last_value, lag, lead

#### 집합 함수
- aggregate functions : sum, avg, min, max, count 

In [2]:
import pyspark

spark = pyspark.sql.SparkSession\
    .builder\
    .master('local')\
    .appName('window_functions')\
    .config(conf=pyspark.SparkConf())\
    .getOrCreate()

21/11/01 09:26:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

In [4]:
_makrsRdd = spark.sparkContext.parallelize(marks).map(lambda x: x.split(', '))
_makrsRdd.collect()

[['김하나', 'English', '100'],
 ['김하나', 'Math', '80'],
 ['임하나', 'English', '70'],
 ['임하나', 'Math', '100'],
 ['김갑돌', 'English', '82.3'],
 ['김갑돌', 'Math', '98.5']]

In [5]:
_markDf = spark.createDataFrame(_makrsRdd, schema=["name", "subject", "mark"])
_markDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)



In [6]:
_markDf.groupBy("subject").count().show()

+-------+-----+
|subject|count|
+-------+-----+
|   Math|    3|
|English|    3|
+-------+-----+



# Group window

column에 대해 group, 즉 윈도우를 만드는 함수를 보자

#### partitionBy()
- column 별로 구분, 즉 컬럼 값에 따라 partition에 포함되는 행을 할당.

- partition을 정하지 않으면 모든 행을 동일한 노드에 할당한다.

#### orderBy()
- partition 내에서 컬럼 값에 대해 행의 순서를 정렬한다.

#### frame()
- 현재 행을 기준으로 포함할 행을 분할

- 행 프레임 : 현재 행을 기준으로 몇 개 앞, 몇 개 뒤의 물리적 범위를 window.rowsBetween(start, end)로 정한다.

- 범위 프레임 : 논리적인 범위를 windowSpec.rangeBetween(start, end)로 정한다.

- 현재 성적이 70점이면 RANGE BETWEEN 20 PERCEDING AND 10 FOLOWING 은 50 - 80을 의미

In [18]:
from pyspark.sql.window import Window

win = Window.partitionBy("subject").orderBy("mark")

### 순위 함수
**row number**

- row_number() 윈도우 함수는 각 그룹별로 일련 번호를 생성한다.

In [9]:
from pyspark.sql.functions import row_number

_markDf.withColumn("row_number", row_number().over(win)).show()

+------+-------+----+----------+
|  name|subject|mark|row_number|
+------+-------+----+----------+
|임하나|   Math| 100|         1|
|김하나|   Math|  80|         2|
|김갑돌|   Math|98.5|         3|
|김하나|English| 100|         1|
|임하나|English|  70|         2|
|김갑돌|English|82.3|         3|
+------+-------+----+----------+



In [12]:
from pyspark.sql.types import FloatType

_markDf = _markDf.withColumn("markF", _markDf['mark'].cast(FloatType()))

In [13]:
from pyspark.sql import functions as F
winF = Window.partitionBy("subject").orderBy(F.col("markF").desc())

In [14]:
_markDf.withColumn("row_number", row_number().over(winF)).show()

+------+-------+----+-----+----------+
|  name|subject|mark|markF|row_number|
+------+-------+----+-----+----------+
|임하나|   Math| 100|100.0|         1|
|김갑돌|   Math|98.5| 98.5|         2|
|김하나|   Math|  80| 80.0|         3|
|김하나|English| 100|100.0|         1|
|김갑돌|English|82.3| 82.3|         2|
|임하나|English|  70| 70.0|         3|
+------+-------+----+-----+----------+



**rank()**
- rank() 윈도우 함수는 각 그룹별로 등위를 계산한다.

In [20]:
from pyspark.sql.functions import rank

_markDf.withColumn("rank", rank().over(winF)).show()

+------+-------+----+-----+----+
|  name|subject|mark|markF|rank|
+------+-------+----+-----+----+
|임하나|   Math| 100|100.0|   1|
|김갑돌|   Math|98.5| 98.5|   2|
|김하나|   Math|  80| 80.0|   3|
|김하나|English| 100|100.0|   1|
|김갑돌|English|82.3| 82.3|   2|
|임하나|English|  70| 70.0|   3|
+------+-------+----+-----+----+



### 분석 함수
**cume_dist()**
- cume_dist() 윈도우 함수는 누적 분포 값을 출력한다.

In [22]:
from pyspark.sql.functions import cume_dist

_markDf.withColumn("cume_dist", cume_dist().over(winF)).show()

+------+-------+----+-----+------------------+
|  name|subject|mark|markF|         cume_dist|
+------+-------+----+-----+------------------+
|임하나|   Math| 100|100.0|0.3333333333333333|
|김갑돌|   Math|98.5| 98.5|0.6666666666666666|
|김하나|   Math|  80| 80.0|               1.0|
|김하나|English| 100|100.0|0.3333333333333333|
|김갑돌|English|82.3| 82.3|0.6666666666666666|
|임하나|English|  70| 70.0|               1.0|
+------+-------+----+-----+------------------+



In [23]:
from pyspark.sql.functions import lag

_markDf.withColumn("lag", lag("mark", 1).over(winF)).show()

+------+-------+----+-----+----+
|  name|subject|mark|markF| lag|
+------+-------+----+-----+----+
|임하나|   Math| 100|100.0|null|
|김갑돌|   Math|98.5| 98.5| 100|
|김하나|   Math|  80| 80.0|98.5|
|김하나|English| 100|100.0|null|
|김갑돌|English|82.3| 82.3| 100|
|임하나|English|  70| 70.0|82.3|
+------+-------+----+-----+----+



In [24]:
from pyspark.sql.functions import lead

_markDf.withColumn("lead", lead("mark", 1).over(winF)).show()

+------+-------+----+-----+----+
|  name|subject|mark|markF|lead|
+------+-------+----+-----+----+
|임하나|   Math| 100|100.0|98.5|
|김갑돌|   Math|98.5| 98.5|  80|
|김하나|   Math|  80| 80.0|null|
|김하나|English| 100|100.0|82.3|
|김갑돌|English|82.3| 82.3|  70|
|임하나|English|  70| 70.0|null|
+------+-------+----+-----+----+



### Aggregate Functions
- 그룹을 구분할 때 정렬할 필요가 없다. mark가 string으로 설정되어 있을 때, 평균, 합계, 최소, 최대 함수는 어떻게 출력될까

- 문제 없이 실행되지만, 결과는 올바르지 않다. 데이터 타입은 항상 주의해야한다.

In [25]:
winAgg = Window.partitionBy("subject")

In [26]:
_markDf.withColumn("avg", F.avg(F.col("markF")).over(winAgg))\
    .withColumn("sum", F.sum(F.col("markF")).over(winAgg))\
    .withColumn("min", F.min(F.col("markF")).over(winAgg))\
    .withColumn("max", F.max(F.col("markF")).over(winAgg))\
    .show()

+------+-------+----+-----+-----------------+-----------------+----+-----+
|  name|subject|mark|markF|              avg|              sum| min|  max|
+------+-------+----+-----+-----------------+-----------------+----+-----+
|김하나|   Math|  80| 80.0|92.83333333333333|            278.5|80.0|100.0|
|임하나|   Math| 100|100.0|92.83333333333333|            278.5|80.0|100.0|
|김갑돌|   Math|98.5| 98.5|92.83333333333333|            278.5|80.0|100.0|
|김하나|English| 100|100.0|84.10000101725261|252.3000030517578|70.0|100.0|
|임하나|English|  70| 70.0|84.10000101725261|252.3000030517578|70.0|100.0|
|김갑돌|English|82.3| 82.3|84.10000101725261|252.3000030517578|70.0|100.0|
+------+-------+----+-----+-----------------+-----------------+----+-----+



In [27]:
spark.stop()